# Used Car Prices CarGurus: KNeighborsRegressor GPU
## Optuna Hyperparameter Optimization

## Set Up Environment, Read Data, Split Train/Test Sets

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/UsedCarsCarGurus/Models/ML/RAPIDS/

/content/drive/MyDrive/UsedCarsCarGurus/Models/ML/RAPIDS


In [ ]:
# Clone RAPIDS-Colab install files and see if GPU is compatible
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 300 (delta 74), reused 99 (delta 55), pack-reused 171
Receiving objects: 100% (300/300), 87.58 KiB | 1.82 MiB/s, done.
Resolving deltas: 100% (136/136), done.
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla P100-PCIE-16GB!
***********************************************************************



In [ ]:
# Update the Colab environment and restart the kernel
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://secu

In [ ]:
# Install CondaColab and restart the kernel 
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:27
🔁 Restarting kernel...


In [ ]:
# See if environment is ready to install RAPIDS
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
# Install RAPIDS using the 'stable' release
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

Found existing installation: cffi 1.14.5
Uninstalling cffi-1.14.5:
  Successfully uninstalled cffi-1.14.5
Found existing installation: cryptography 3.4.5
Uninstalling cryptography-3.4.5:
  Successfully uninstalled cryptography-3.4.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Installing RAPIDS Stable 21.12
Starting the RAPIDS install on Colab.  This will take about 15 minutes.
Solving environment: ...working... WARNING conda.core.solve:_add_specs(611): pinned spec cudatoolkit=11.1 conflicts with explicit specs.  Overriding pinned spec.
failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... WARNING conda.core.solve:_add_specs(611): pinned spec cudatoolkit=11.1 conflicts with explicit specs.  Overriding pinned spec.
failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... WARNING conda.core.solve:_add_specs(611): pinned spec c

In [ ]:
# Install/import dependencies
!pip install optuna
!pip install dask_optuna
import os
import random
import numpy as np
import time
from contextlib import contextmanager
import cupy as cp
import cudf
import cuml
from cuml.model_selection import train_test_split
from cuml.neighbors import KNeighborsRegressor
import optuna
from optuna import Trial
import dask
import dask_cudf
import dask_optuna
from dask_cuda import LocalCUDACluster
from dask.distributed import Client, wait, performance_report
import joblib
from joblib import parallel_backend
import pickle
from datetime import datetime, timedelta
from timeit import default_timer as timer
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.offline as py
import matplotlib.pyplot as plt
import seaborn as sns
optuna.logging.set_verbosity(optuna.logging.WARNING) 
my_dpi = 96
print('\n')
!/usr/local/cuda/bin/nvcc --version
!nvidia-smi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 5.1 MB/s 
     |████████████████████████████████| 81 kB 9.7 MB/s 
     |████████████████████████████████| 209 kB 36.9 MB/s 
     |████████████████████████████████| 1.6 MB 47.5 MB/s 
     |████████████████████████████████| 150 kB 57.7 MB/s 
     |████████████████████████████████| 78 kB 6.1 MB/s 
     |████████████████████████████████| 49 kB 5.0 MB/s 
     |████████████████████████████████| 112 kB 52.9 MB/s 
     |████████████████████████████████| 146 kB 39.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=9593925913f5fb0ad45302e8aecf3624d7678a3aff5e8c30f809d73551bcdb5c
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/sim

In [ ]:
# Define function to time code blocks
@contextmanager
def timed(name):
    t0 = time.time()
    yield
    t1 = time.time()
    print('..%-24s:  %8.4f' % (name, t1 - t0))

In [ ]:
# Set up CUDA cluster
cluster = LocalCUDACluster(threads_per_worker=1, ip='', 
                           dashboard_address='8081') 
c = Client(cluster)

# Query the client for all connected workers
workers = c.has_what().keys()
n_workers = len(workers)
c

/usr/local/lib/python3.7/site-packages/distributed/client.py:1131: VersionMismatchWarning:

Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| tornado | 5.1.1  | 5.1.1     | 6.1     |
+---------+--------+-----------+---------+



Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://172.28.0.2:8081/status,
Dashboard: http://172.28.0.2:8081/status,Workers: 1
Total threads: 1,Total memory: 25.46 GiB
Status: running,Using processes: True
Comm: tcp://172.28.0.2:45339,Workers: 1
Dashboard: http://172.28.0.2:8081/status,Total threads: 1
Started: Just now,Total memory: 25.46 GiB
Comm: tcp://172.28.0.2:35409,Total threads: 1
Dashboard: http://172.28.0.2:37745/status,Memory: 25.46 GiB
Nanny: tcp://172.28.0.2:36179,


In [ ]:
# Set seed 
seed_value = 42
os.environ['KNR_GPU'] = str(seed_value)
cp.random.seed(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)

In [ ]:
%cd /content/drive/MyDrive/UsedCarsCarGurus/Data/

/content/drive/MyDrive/UsedCarsCarGurus/Data


In [ ]:
# Read data
df = cudf.read_csv('usedCars_final.csv')
print('Number of rows and columns:', df.shape)

Number of rows and columns: (327912, 28)


In [ ]:
# Find categorical variables
cat_cols = list(df.select_dtypes('object').columns)

# Set up features and target
X, y = df.drop(['price'], axis=1), df['price'].astype('float32')

## 100 Trials

In [ ]:
# Write results to ML Results
%cd /content/drive/MyDrive/UsedCarsCarGurus/Models/ML/KNR/Optuna/Model_PKL/

# Define a name for the trials
study_name = 'dask_optuna_knr_rmse_tpe'

/content/drive/MyDrive/UsedCarsCarGurus/Models/ML/KNR/Optuna/Model_PKL


In [ ]:
# Define function to train/evaluate the model
def train_and_eval(X_param, y_param, n_neighbors=10, 
                   metric='euclidean', verbose=False): 
    """
        Partition data into train/test sets, train and evaluate the model
        for the given parameters.
        
        Params
        ______
        
        X_param:  DataFrame. 
                  The data to use for training and testing. 
        y_param:  Series. 
                  The label for training

        Returns
        score: RMSE of the fitted model
    """
    # Prepare the data for modeling
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        y, 
                                                        random_state=seed_value)
    X_train = cudf.get_dummies(X_train, columns=cat_cols) 
    X_train = X_train.astype('float32')
    X_test = cudf.get_dummies(X_test, columns=cat_cols)
    X_test = X_test.astype('float32')

    # Define model
    model = KNeighborsRegressor(n_neighbors=n_neighbors, metric=metric,
                                verbose=verbose)
    
    # Start timer for each trial
    start = timer()

    # Fit model
    model.fit(X_train, y_train)
    run_time = timer() - start

    # Predict on the model
    y_pred = model.predict(X_test)
    score = mean_squared_error(y_test.to_numpy(), y_pred.to_numpy(), 
                               squared=False)     
    print('- Trial RMSE:', score)
    
    return score

In [ ]:
# Fit model with default parameters
print('Score with default parameters : ', train_and_eval(X, y))

- Trial RMSE: 4896.987
Score with default parameters :  4896.987


In [ ]:
# Define objective function with the parameters to be tested
def objective(trial, X_param, y_param):
    
    joblib.dump(study, 'KNR_Optuna_100_GPU.pkl')
    
    # Search parameters
    n_neighbors = trial.suggest_int('n_neighbors', 3, 50) 
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 
                                                  'chebyshev', 'minkowski'])

    score = train_and_eval(X_param, y_param,
                           n_neighbors=n_neighbors, 
                           verbose=False)

    return score

In [ ]:
with timed('dask_optuna'):
    # Begin HPO trials 
    # Start timer for experiment
    start_time = datetime.now()
    print('%-20s %s' % ('Start Time', start_time))
    if os.path.isfile('KNR_Optuna_100_GPU.pkl'): 
      study = joblib.load('KNR_Optuna_100_GPU.pkl')
    else: 
      study = optuna.create_study(sampler=optuna.samplers.TPESampler(), 
                                  study_name=study_name,
                                  direction='minimize')
      
    # Optimize in parallel on Dask cluster
    with parallel_backend('dask'):
        study.optimize(lambda trial: objective(trial, X, y), 
                       n_trials=100,
                       n_jobs=n_workers)

# End timer for experiment
end_time = datetime.now()
print('%-20s %s' % ('Start Time', start_time))
print('%-20s %s' % ('End Time', end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))
print('\n')
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Lowest RMSE', study.best_value)

Start Time           2022-06-04 22:27:09.733177
- Trial RMSE: 5023.282
- Trial RMSE: 4902.1064
- Trial RMSE: 4990.168
- Trial RMSE: 5152.733
- Trial RMSE: 4998.132
- Trial RMSE: 5152.733
- Trial RMSE: 4906.328
- Trial RMSE: 4914.573
- Trial RMSE: 4935.5347
- Trial RMSE: 5047.3623
- Trial RMSE: 5072.706
- Trial RMSE: 4902.1064
- Trial RMSE: 4896.4062
- Trial RMSE: 4896.987
- Trial RMSE: 4896.6997
- Trial RMSE: 4954.6313
- Trial RMSE: 4899.4194
- Trial RMSE: 4899.4194
- Trial RMSE: 4935.5347
- Trial RMSE: 4963.2544
- Trial RMSE: 4896.6997
- Trial RMSE: 4896.987
- Trial RMSE: 4977.241
- Trial RMSE: 4923.4565
- Trial RMSE: 4897.4077
- Trial RMSE: 4914.308
- Trial RMSE: 4914.573
- Trial RMSE: 4985.437
- Trial RMSE: 4896.4062
- Trial RMSE: 5013.4805
- Trial RMSE: 4948.772
- Trial RMSE: 4896.4062
- Trial RMSE: 4897.4077
- Trial RMSE: 4897.4077
- Trial RMSE: 4977.241
- Trial RMSE: 4914.573
- Trial RMSE: 4977.241
- Trial RMSE: 4906.328
- Trial RMSE: 4897.4077
- Trial RMSE: 4923.4565
- Trial RMS

In [ ]:
# Write results to ML Results
%cd /content/drive/MyDrive/UsedCarsCarGurus/Models/ML/KNR/Optuna/trialOptions/

/content/drive/MyDrive/UsedCarsCarGurus/Models/ML/KNR/Optuna/trialOptions


In [ ]:
# Output from HPO trials
trials_df = study.trials_dataframe()
trials_df.rename(columns={'number': 'iteration'}, inplace=True)
trials_df.rename(columns={'value': 'rmse'},inplace=True)
trials_df.rename(columns={'params_metric': 'metric'}, inplace=True)
trials_df.rename(columns={'params_n_neighbors': 'n_neighbors'}, inplace=True)
trials_df = trials_df.sort_values('rmse', ascending=True)
print(trials_df) 

# Write processed data to csv
trials_df.to_csv('KNR_Optuna_100_GPU.csv', index=False)

    iteration         rmse             datetime_start  \
31         31  4896.406250 2022-06-04 22:27:43.237748   
70         70  4896.406250 2022-06-04 22:28:25.292005   
83         83  4896.406250 2022-06-04 22:28:39.422489   
71         71  4896.406250 2022-06-04 22:28:26.370286   
12         12  4896.406250 2022-06-04 22:27:22.796066   
..        ...          ...                        ...   
98         98  5076.933105 2022-06-04 22:28:55.816185   
47         47  5152.732910 2022-06-04 22:28:00.401556   
58         58  5152.732910 2022-06-04 22:28:12.314977   
5           5  5152.732910 2022-06-04 22:27:15.188892   
3           3  5152.732910 2022-06-04 22:27:13.051714   

            datetime_complete               duration     metric  n_neighbors  \
31 2022-06-04 22:27:44.310128 0 days 00:00:01.072380  minkowski           11   
70 2022-06-04 22:28:26.370115 0 days 00:00:01.078110  minkowski           11   
83 2022-06-04 22:28:40.502530 0 days 00:00:01.080041  minkowski           1

In [ ]:
# Write results to ML Results
%cd /content/drive/MyDrive/UsedCarsCarGurus/Models/ML/KNR/Optuna/Model_Explanations/

/content/drive/MyDrive/UsedCarsCarGurus/Models/ML/KNR/Optuna/Model_Explanations


In [ ]:
# Plot_optimization_history: shows the scores from all trials as well as the best score so far at each point.
fig = optuna.visualization.plot_optimization_history(study)
py.plot(fig, filename='optimizationHistory_KNR_Optuna_100_GPU.pkl.html')
fig.show()

In [ ]:
# plot_parallel_coordinate: interactively visualizes the hyperparameters and scores
fig = optuna.visualization.plot_parallel_coordinate(study)
py.plot(fig, filename='parallelCoordinate_KNR_Optuna_100_GPU.pkl.html')
fig.show()

In [ ]:
# plot_slice: shows the change of the hyperparamters space over the search. 
fig = optuna.visualization.plot_slice(study)
py.plot(fig, filename='slice_KNR_Optuna_100_GPU.pkl.html')
fig.show()

In [ ]:
# Visualize empirical distribution function
fig = optuna.visualization.plot_edf(study)
py.plot(fig, filename='edf_KNR_Optuna_100_GPU.html')
fig.show()

In [ ]:
# Arrange best parameters to fit model for model metrics
params = study.best_params   
params

{'metric': 'chebyshev', 'n_neighbors': 11}

In [ ]:
# Write results to ML Results
%cd /content/drive/MyDrive/UsedCarsCarGurus/Models/ML/KNR/Optuna/Model_PKL/

/content/drive/MyDrive/UsedCarsCarGurus/Models/ML/KNR/Optuna/Model_PKL


In [ ]:
# Set up train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    random_state=seed_value)

In [ ]:
X_train = cudf.get_dummies(X_train, columns=cat_cols)
X_train = X_train.astype('float32')

In [ ]:
X_test = cudf.get_dummies(X_test, columns=cat_cols)
X_test = X_test.astype('float32')

In [ ]:
# Re-create the best model and train on the training data
#best_model = KNeighborsRegressor(**params)
best_model = KNeighborsRegressor(n_neighbors=11, metric='chebyshev')

# Fit the model
best_model.fit(X_train, y_train)

# Save model
Pkl_Filename = 'KNR_Optuna_trials100_GPU.pkl'  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(best_model, file)

# =============================================================================
# # To load saved model
# model = joblib.load('KNR_Optuna_trials100_GPU.pkl')
# print(model)
# =============================================================================

In [ ]:
print('\nModel Metrics for KNR HPO 100 GPU trials')
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

print('MAE train: %.3f, test: %.3f' % (
        mean_absolute_error(y_train.to_numpy(), y_train_pred.to_numpy()),
        mean_absolute_error(y_test.to_numpy(), y_test_pred.to_numpy())))
print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train.to_numpy(), y_train_pred.to_numpy()),
        mean_squared_error(y_test.to_numpy(), y_test_pred.to_numpy())))
print('RMSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train.to_numpy(), y_train_pred.to_numpy(), 
                           squared=False),
        mean_squared_error(y_test.to_numpy(), y_test_pred.to_numpy(), 
                           squared=False)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train.to_numpy(), y_train_pred.to_numpy()),
        r2_score(y_test.to_numpy(), y_test_pred.to_numpy())))


Model Metrics for KNR HPO 100 GPU trials
MAE train: 3436.407, test: 3784.558
MSE train: 21261224.000, test: 25787092.000
RMSE train: 4610.989, test: 5078.099
R^2 train: 0.768, test: 0.717


In [ ]:
# Evaluate on the testing data 
print('The best model from optimization scores {:.5f} MSE on the test set.'.format(mean_squared_error(y_test.to_numpy(), 
                                                                                                      y_test_pred.to_numpy())))
print('This was achieved using these conditions:')
print(trials_df.iloc[0])

The best model from optimization scores 25787092.00000 MSE on the test set.
This was achieved using these conditions:
iteration                                    31
rmse                                 4896.40625
datetime_start       2022-06-04 22:27:43.237748
datetime_complete    2022-06-04 22:27:44.310128
duration                 0 days 00:00:01.072380
metric                                minkowski
n_neighbors                                  11
state                                  COMPLETE
Name: 31, dtype: object


## 1000 Trials

In [ ]:
# Write results to ML Results
%cd /content/drive/MyDrive/UsedCarsCarGurus/Models/ML/KNR/Optuna/Model_PKL/

/content/drive/MyDrive/UsedCarsCarGurus/Models/ML/KNR/Optuna/Model_PKL


In [ ]:
def objective(trial, X_param, y_param):
    
    joblib.dump(study, 'KNR_Optuna_1000_GPU.pkl')
    
    # Parameter searcg
    n_neighbors = trial.suggest_int('n_neighbors', 3, 50) 
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 
                                                  'chebyshev', 'minkowski'])

    score = train_and_eval(X_param, y_param,
                           n_neighbors=n_neighbors, 
                           verbose=False)

    return score

In [ ]:
with timed('dask_optuna'):
    # Begin HPO trials 
    # Start timer for experiment
    start_time = datetime.now()
    print('%-20s %s' % ('Start Time', start_time))
    if os.path.isfile('KNR_Optuna_1000_GPU.pkl'): 
      study = joblib.load('KNR_Optuna_1000_GPU.pkl')
    else: 
      study = optuna.create_study(sampler=optuna.samplers.TPESampler(), 
                                  study_name=study_name,
                                  direction='minimize')
      
    # Optimize in parallel on your Dask cluster
    with parallel_backend('dask'):
        study.optimize(lambda trial: objective(trial, X, y), 
                       n_trials=1000,
                       n_jobs=n_workers)

# End timer for experiment
end_time = datetime.now()
print('%-20s %s' % ('Start Time', start_time))
print('%-20s %s' % ('End Time', end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))
print('\n')
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Lowest RMSE', study.best_value)

Start Time           2022-06-04 22:36:42.774058
- Trial RMSE: 4914.573
- Trial RMSE: 5068.23
- Trial RMSE: 4918.964
- Trial RMSE: 4937.2476
- Trial RMSE: 5028.251
- Trial RMSE: 4994.4575
- Trial RMSE: 5063.524
- Trial RMSE: 5038.556
- Trial RMSE: 5051.233
- Trial RMSE: 4896.987
- Trial RMSE: 5152.733
- Trial RMSE: 4908.732
- Trial RMSE: 4906.328
- Trial RMSE: 4896.4062
- Trial RMSE: 4896.6997
- Trial RMSE: 4954.6313
- Trial RMSE: 4969.6724
- Trial RMSE: 4896.4062
- Trial RMSE: 4954.6313
- Trial RMSE: 4897.4077
- Trial RMSE: 5152.733
- Trial RMSE: 4896.6997
- Trial RMSE: 4935.5347
- Trial RMSE: 4897.4077
- Trial RMSE: 4914.308
- Trial RMSE: 4985.437
- Trial RMSE: 4897.4077
- Trial RMSE: 4928.223
- Trial RMSE: 4937.2476
- Trial RMSE: 4908.732
- Trial RMSE: 5004.2554
- Trial RMSE: 4896.6997
- Trial RMSE: 4902.1064
- Trial RMSE: 4896.6997
- Trial RMSE: 4923.4565
- Trial RMSE: 4937.2476
- Trial RMSE: 5152.733
- Trial RMSE: 4897.9854
- Trial RMSE: 4899.4194
- Trial RMSE: 4896.4062
- Trial RM

In [ ]:
# Write results to ML Results
%cd /content/drive/MyDrive/UsedCarsCarGurus/Models/ML/KNR/Optuna/trialOptions/

/content/drive/MyDrive/UsedCarsCarGurus/Models/ML/KNR/Optuna/trialOptions


In [ ]:
# Output from HPO trials
trials_df = study.trials_dataframe()
trials_df.rename(columns={'number': 'iteration'}, inplace=True)
trials_df.rename(columns={'value': 'rmse'},inplace=True)
trials_df.rename(columns={'params_metric': 'metric'}, inplace=True)
trials_df.rename(columns={'params_n_neighbors': 'n_neighbors'}, inplace=True)
trials_df = trials_df.sort_values('rmse', ascending=True)
print(trials_df) 

# Write processed data to csv
trials_df.to_csv('KNR_Optuna_1000_GPU.csv', index=False)

     iteration        rmse             datetime_start  \
175        175  4896.40625 2022-06-04 22:39:52.621211   
160        160  4896.40625 2022-06-04 22:39:36.056961   
770        770  4896.40625 2022-06-04 22:51:41.374032   
162        162  4896.40625 2022-06-04 22:39:38.224020   
163        163  4896.40625 2022-06-04 22:39:39.318820   
..         ...         ...                        ...   
396        396  5152.73291 2022-06-04 22:44:03.938120   
36          36  5152.73291 2022-06-04 22:37:22.089024   
703        703  5152.73291 2022-06-04 22:50:16.495416   
20          20  5152.73291 2022-06-04 22:37:04.877784   
10          10  5152.73291 2022-06-04 22:36:54.070525   

             datetime_complete               duration     metric  n_neighbors  \
175 2022-06-04 22:39:53.728639 0 days 00:00:01.107428  minkowski           11   
160 2022-06-04 22:39:37.138032 0 days 00:00:01.081071  minkowski           11   
770 2022-06-04 22:51:42.875439 0 days 00:00:01.501407  manhattan        

In [ ]:
# Write results to ML Results
%cd /content/drive/MyDrive/UsedCarsCarGurus/Models/ML/KNR/Optuna/Model_Explanations/

/content/drive/MyDrive/UsedCarsCarGurus/Models/ML/KNR/Optuna/Model_Explanations


In [ ]:
# Plot_optimization_history: shows the scores from all trials as well as the best score so far at each point.
fig = optuna.visualization.plot_optimization_history(study)
py.plot(fig, filename='optimizationHistory_KNR_Optuna_1000_GPU.html')
fig.show()

In [ ]:
# plot_parallel_coordinate: interactively visualizes the hyperparameters and scores
fig = optuna.visualization.plot_parallel_coordinate(study)
py.plot(fig, filename='parallelCoordinate_KNR_Optuna_1000_GPU.html')
fig.show()

In [ ]:
# plot_slice: shows the change of the hyperparamters space over the search. 
fig = optuna.visualization.plot_slice(study)
py.plot(fig, filename='slice_KNR_Optuna_1000_GPU.html')
fig.show()

In [ ]:
# Visualize empirical distribution function
fig = optuna.visualization.plot_edf(study)
py.plot(fig, filename='edf_KNR_Optuna_1000_GPU.html')
fig.show()

In [ ]:
# Arrange best parameters to fit model for model metrics
params = study.best_params   
params

{'metric': 'manhattan', 'n_neighbors': 11}

In [ ]:
# Write results to ML Results
%cd /content/drive/MyDrive/UsedCarsCarGurus/Models/ML/KNR/Optuna/Model_PKL/

/content/drive/MyDrive/UsedCarsCarGurus/Models/ML/KNR/Optuna/Model_PKL


In [ ]:
# Set up train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    random_state=seed_value)

In [ ]:
X_train = cudf.get_dummies(X_train, columns=cat_cols)
X_train = X_train.astype('float32')

In [ ]:
X_test = cudf.get_dummies(X_test, columns=cat_cols)
X_test = X_test.astype('float32')

In [ ]:
# Re-create the best model and train on the training data
best_model = KNeighborsRegressor(n_neighbors=11, metric='manhattan')

# Fit the model
best_model.fit(X_train, y_train)

# Save model
Pkl_Filename = 'KNR_Optuna_trials1000_GPU_man.pkl'  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(best_model, file)

# =============================================================================
# # To load saved model
# model = joblib.load('KNR_Optuna_trials1000_GPU_man.pkl')
# print(model)
# =============================================================================

In [ ]:
print('\nModel Metrics for KNR HPO 1000 GPU trials - Manhattan')
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

print('MAE train: %.3f, test: %.3f' % (
        mean_absolute_error(y_train.to_numpy(), y_train_pred.to_numpy()),
        mean_absolute_error(y_test.to_numpy(), y_test_pred.to_numpy())))
print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train.to_numpy(), y_train_pred.to_numpy()),
        mean_squared_error(y_test.to_numpy(), y_test_pred.to_numpy())))
print('RMSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train.to_numpy(), y_train_pred.to_numpy(), 
                           squared=False),
        mean_squared_error(y_test.to_numpy(), y_test_pred.to_numpy(), 
                           squared=False)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train.to_numpy(), y_train_pred.to_numpy()),
        r2_score(y_test.to_numpy(), y_test_pred.to_numpy())))


Model Metrics for KNR HPO 1000 GPU trials
MAE train: 3080.647, test: 3387.940
MSE train: 17225190.000, test: 20828008.000
RMSE train: 4150.324, test: 4563.771
R^2 train: 0.812, test: 0.772


In [ ]:
# Re-create the best model and train on the training data
# Tie for lowest RMSE
best_model = KNeighborsRegressor(n_neighbors=11, metric='minkowski')

# Fit the model
best_model.fit(X_train, y_train)

# Save model
Pkl_Filename = 'KNR_Optuna_trials1000_GPU_min.pkl'  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(best_model, file)

# =============================================================================
# # To load saved model
# model = joblib.load('KNR_Optuna_trials1000_GPU_min.pkl')
# print(model)
# =============================================================================

In [ ]:
print('\nModel Metrics for KNR HPO 1000 GPU trials - Minkowski')
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

print('MAE train: %.3f, test: %.3f' % (
        mean_absolute_error(y_train.to_numpy(), y_train_pred.to_numpy()),
        mean_absolute_error(y_test.to_numpy(), y_test_pred.to_numpy())))
print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train.to_numpy(), y_train_pred.to_numpy()),
        mean_squared_error(y_test.to_numpy(), y_test_pred.to_numpy())))
print('RMSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train.to_numpy(), y_train_pred.to_numpy(), 
                           squared=False),
        mean_squared_error(y_test.to_numpy(), y_test_pred.to_numpy(), 
                           squared=False)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train.to_numpy(), y_train_pred.to_numpy()),
        r2_score(y_test.to_numpy(), y_test_pred.to_numpy())))


Model Metrics for KNR HPO 1000 GPU trials - Minkowski
MAE train: 3313.156, test: 3648.266
MSE train: 19763276.000, test: 23974796.000
RMSE train: 4445.591, test: 4896.406
R^2 train: 0.784, test: 0.737


In [ ]:
# Evaluate on the testing data 
print('The best model from optimization scores {:.5f} MSE on the test set.'.format(mean_squared_error(y_test.to_numpy(), 
                                                                                                      y_test_pred.to_numpy())))
print('This was achieved using these conditions:')
print(trials_df.iloc[0])

The best model from optimization scores 20828008.00000 MSE on the test set.
This was achieved using these conditions:
iteration                                   175
rmse                                 4896.40625
datetime_start       2022-06-04 22:39:52.621211
datetime_complete    2022-06-04 22:39:53.728639
duration                 0 days 00:00:01.107428
metric                                minkowski
n_neighbors                                  11
state                                  COMPLETE
Name: 175, dtype: object


In [ ]:
# Re-create the best model and train on the training data
# Test other distance
best_model = KNeighborsRegressor(n_neighbors=11, metric='euclidean')

# Fit the model
best_model.fit(X_train, y_train)

# Save model
Pkl_Filename = 'KNR_Optuna_trials1000_GPU_euc.pkl'  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(best_model, file)

# =============================================================================
# # To load saved model
# model = joblib.load('KNR_Optuna_trials1000_GPU_euc.pkl')
# print(model)
# =============================================================================

In [ ]:
print('\nModel Metrics for KNR HPO 1000 GPU trials - Euclidean')
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

print('MAE train: %.3f, test: %.3f' % (
        mean_absolute_error(y_train.to_numpy(), y_train_pred.to_numpy()),
        mean_absolute_error(y_test.to_numpy(), y_test_pred.to_numpy())))
print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train.to_numpy(), y_train_pred.to_numpy()),
        mean_squared_error(y_test.to_numpy(), y_test_pred.to_numpy())))
print('RMSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train.to_numpy(), y_train_pred.to_numpy(), 
                           squared=False),
        mean_squared_error(y_test.to_numpy(), y_test_pred.to_numpy(), 
                           squared=False)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train.to_numpy(), y_train_pred.to_numpy()),
        r2_score(y_test.to_numpy(), y_test_pred.to_numpy())))


Model Metrics for KNR HPO 1000 GPU trials - Euclidean
MAE train: 3313.156, test: 3648.266
MSE train: 19763276.000, test: 23974796.000
RMSE train: 4445.591, test: 4896.406
R^2 train: 0.784, test: 0.737


In [ ]:
# Re-create the best model and train on the training data
# Test other distance
best_model = KNeighborsRegressor(n_neighbors=11, metric='chebyshev')

# Fit the model
best_model.fit(X_train, y_train)

# Save model
Pkl_Filename = 'KNR_Optuna_trials1000_GPU_cheb.pkl'  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(best_model, file)

# =============================================================================
# # To load saved model
# model = joblib.load('KNR_Optuna_trials1000_GPU_cheb.pkl')
# print(model)
# =============================================================================

In [ ]:
print('\nModel Metrics for KNR HPO 1000 GPU trials - Chebyshev')
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

print('MAE train: %.3f, test: %.3f' % (
        mean_absolute_error(y_train.to_numpy(), y_train_pred.to_numpy()),
        mean_absolute_error(y_test.to_numpy(), y_test_pred.to_numpy())))
print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train.to_numpy(), y_train_pred.to_numpy()),
        mean_squared_error(y_test.to_numpy(), y_test_pred.to_numpy())))
print('RMSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train.to_numpy(), y_train_pred.to_numpy(), 
                           squared=False),
        mean_squared_error(y_test.to_numpy(), y_test_pred.to_numpy(),
                           squared=False)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train.to_numpy(), y_train_pred.to_numpy()),
        r2_score(y_test.to_numpy(), y_test_pred.to_numpy())))


Model Metrics for KNR HPO 1000 GPU trials - Chebyshev
MAE train: 3436.407, test: 3784.558
MSE train: 21261224.000, test: 25787092.000
RMSE train: 4610.989, test: 5078.099
R^2 train: 0.768, test: 0.717
